## Imports

In [ ]:
import pandas as pd
from datetime import datetime
from service import clean_content, get_content, split_content_by_type_weather, transform_metar_file, combine_csv_files_with_metar, get_engine
import os

## Request Data

In [ ]:
# Variables

airport = 'UGSB'
from_year = 2020
to_year = datetime.now().year

In [ ]:
content = get_content(airport=airport, from_year=from_year)

cleaned_content = clean_content(content)

data = split_content_by_type_weather(cleaned_content)

In [ ]:
metar = pd.DataFrame(data.get('METAR', []), columns=['metar'])
taf = pd.DataFrame(data.get('TAF', []), columns=['taf'])

In [ ]:
metar.head()

In [ ]:
taf.head()

In [ ]:
metar.to_csv(f'./src/{airport}_metar_{from_year}_{to_year}.csv', index=False)

taf.to_csv(f'./src/{airport}_taf_{from_year}_{to_year}.csv', index=False)

## Prepare Data

In [ ]:
folder_path = 'src'
folder_to_save = './datasets/'

columns_to_save = ['datetime', 'airport', 'wind_direct', 'wind_power',
                   'wind_gust', 'wind_uom', 'wind_variable', 'temp', 'dew_point', 'metar']

metar_files = [os.path.join(folder_path, file) for file in os.listdir(
    folder_path) if 'metar' in file.lower() and file.endswith('.csv')]
taf_files = [os.path.join(folder_path, file) for file in os.listdir(
    folder_path) if 'taf' in file.lower() and file.endswith('.csv')]


for metar_file in metar_files:
    df_metar = transform_metar_file(metar_file)
    df_metar.to_csv(
        f'{os.path.join(folder_to_save, os.path.basename(metar_file))}', index=False)

## Upload to Database

In [ ]:
folder = './datasets/'
df = combine_csv_files_with_metar(folder).drop_duplicates()

In [ ]:
engine = get_engine('weather')

df.to_sql('metar', engine, if_exists='replace', index=False)